In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import keras as ks
import numpy as np
import pandas as pd

In [2]:
train_data_filename = 'EMNIST_data/emnist-balanced-train.csv'
test_data_filename = 'EMNIST_data/emnist-balanced-test.csv'
raw_train_data = pd.read_csv(train_data_filename, header=None).values
raw_test_data = pd.read_csv(test_data_filename, header=None).values

num_classes = 47
eye = np.eye(num_classes)
x_train, y_train = raw_train_data[:, 1:].astype(np.float32), eye[raw_train_data[:, :1].ravel()]
x_test, y_test = raw_test_data[:, 1:].astype(np.float32), eye[raw_test_data[:, :1].ravel()]

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print(x_train.shape, x_test.shape)

x_train /= 255.0
x_test /= 255.0

(112800, 28, 28, 1) (18800, 28, 28, 1)


In [3]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28, 1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=ks.losses.categorical_crossentropy,
              optimizer=ks.optimizers.Adadelta(),
              metrics=['accuracy'])

In [4]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 112800 samples, validate on 18800 samples
Epoch 1/20
112800/112800 [==============================] - 23s 207us/step - loss: 1.1669 - acc: 0.6560 - val_loss: 0.5300 - val_acc: 0.8295
Epoch 2/20
  8576/112800 [=>............................] - ETA: 18s - loss: 0.7330 - acc: 0.7657

KeyboardInterrupt: 

In [43]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.39517347572331735
Test accuracy: 0.878563829787234


In [44]:
# save model
model.save('model/emnist.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               1179776   
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
__________